In [1]:
import os
import sys
import time
nb_dir = os.path.split(os.getcwd())[0]

sys.path.append(nb_dir)
sys.path.append(os.path.split(nb_dir)[0])

In [2]:
import pandas as pd
from PIL import Image, ImageDraw
import copy

In [3]:
from services.xml_document_info import (get_xml_info, get_xml_image_info)
from services.get_xml import  create_pdf_processing_paths, extract_pdf_metadata, process_input_pdf
from anuvaad_auditor.loghandler import log_info
from anuvaad_auditor.loghandler import log_error
from src.services import main
from services import get_xml
from src.services.child_text_unify_to_parent import ChildTextUnify
from services.preprocess import prepocess_pdf_regions
from services.get_tables import page_num_correction , get_text_table_line_df
from src.services.ocr_text_utilities import  tesseract_ocr

from utilities.xml_utils import (extract_image_paths_from_pdf,
                                 extract_xml_from_digital_pdf,
                                 extract_html_bg_images_from_digital_pdf,
                                 create_directory,
                                 read_directory_files)
import config

In [4]:
def draw_box(filepath, desired_width, desired_height, df, color="green", save=False):
    image  = Image.open(filepath)
    image  = image.resize((desired_width, desired_height))
    draw   = ImageDraw.Draw(image)
    
    for index, row in df.iterrows():
        left   = int(row['text_left'])
        right  = int(row['text_width'] + left)
        top    = int(row['text_top'])
        bottom = int(row["text_height"] + top)
        
        draw.rectangle(((left, top), (right,bottom)), outline=color)
    save_filepath = os.path.join(os.path.dirname(filepath), 'processed_' + os.path.basename(filepath))
    if save:
        image.save(save_filepath)
    
    return image

def show_df(df):
    return df.head(df.shape[0])

## start of the program

In [5]:
'''
  folder structure of test data goes like this
  - notebooks
      - sample-data
          - input
          - output
          
  the pdfs are present in "input" directory and they are *.pdf is added into .gitignore
  just to save repo size.
'''

base_dir   = os.getcwd()
input_dir  = os.path.join(base_dir, 'sample-data','input')
save_dir = os.path.join(base_dir, 'sample-data', 'bbox_output')

output_dir = os.path.join(base_dir, 'sample-data', 'output')
create_directory(input_dir)
create_directory(output_dir)

filename   = '219200009452014_1.pdf'

## extract the pdf into xml & background images for downstream processing

In [6]:
pdf_filepath      = os.path.join(input_dir, filename)
working_dir       = os.path.join(output_dir, os.path.splitext(filename)[0])

ret               = create_directory(working_dir)

start_time        = time.time()

pdf_image_paths   = extract_image_paths_from_pdf(pdf_filepath, working_dir)
pdf_xml_dir       = extract_xml_from_digital_pdf(pdf_filepath, working_dir)
pdf_bg_image_dir  = extract_html_bg_images_from_digital_pdf(pdf_filepath, working_dir)

end_time          = time.time()
extraction_time   = end_time - start_time

In [7]:
xml_files      = read_directory_files(pdf_xml_dir, pattern='*.xml')
bg_files       = read_directory_files(pdf_bg_image_dir, pattern='*.png')

print("Extracted xml, background images & images of file: %s" % (filename))
print('Extraction time (%f) average extraction time (%f)' % (extraction_time, extraction_time/len(bg_files)))

Extracted xml, background images & images of file: 219200009452014_1.pdf
Extraction time (1.562162) average extraction time (0.156216)


In [8]:
xml_dfs, page_width, page_height = get_xml_info(xml_files[0])
img_dfs, page_width, page_height = get_xml_image_info(xml_files[0])
pages = len(xml_dfs)

Total number of pages (10) in file (219200009452014_1.xml)
Total number of pages (10) in file (219200009452014_1.xml)


# Draw Bounding box using dfs

In [9]:
header_region, footer_region = prepocess_pdf_regions(xml_dfs, page_height)

/home/naresh/Tarento/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/services/preprocess.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['text_right']  = sub_df['text_left'] + sub_df['text_width']
/home/naresh/Tarento/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/services/preprocess.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['text_bottom'] = sub_df['text_top'] + sub_df['text_height']


In [10]:
def doc_structure_analysis(pages,xml_dfs,img_dfs,working_dir,header_region , footer_region, multiple_pages):
    
    '''
        Document structure analysis to get:
            - in_dfs
            - table_dfs
            - line_dfs
            - h_dfs
            - v_dfs
            - p_dfs
            - text_block_dfs

    '''
    log_info("Service main", "document structure analysis started  ===>", None)
    
    text_merger = ChildTextUnify()
    in_dfs, table_dfs, line_dfs = get_text_table_line_df(pages,working_dir, xml_dfs,img_dfs,None)
    h_dfs          = get_xml.get_hdfs(pages, in_dfs, config.DOCUMENT_CONFIGS,header_region , footer_region, multiple_pages)
    v_dfs          = get_xml.get_vdfs(pages, h_dfs, config.DOCUMENT_CONFIGS)
    p_dfs          = get_xml.get_pdfs(pages, v_dfs, config.DOCUMENT_CONFIGS, config.BLOCK_CONFIGS)
    #text_block_dfs = text_merger.unify_child_text_blocks(pages, p_dfs, config.DROP_TEXT)

    log_info("Service main", "document structure analysis successfully completed", None)

    return in_dfs,h_dfs, v_dfs, p_dfs

In [11]:
in_dfs,h_dfs, v_dfs, p_dfs = doc_structure_analysis(pages,xml_dfs,img_dfs,working_dir,header_region , footer_region, True)

UnboundLocalError: local variable 'lines' referenced before assignment

In [ ]:
"\u0905\u092a\u0940\u0932"

# # OCR for vernacular language

In [21]:
### OCR for class 1 vernacular documents
ocr_dfs  = tesseract_ocr(pdf_image_paths, page_width, page_height, p_dfs, "en" )

In [22]:
page_index      = 0
img_filepath    = pdf_image_paths[page_index]

in_df   = in_dfs[page_index]
h_df    = h_dfs[page_index]
v_df    = v_dfs[page_index]
p_df    = p_dfs[page_index]
ocr_df  = ocr_dfs[page_index]

In [23]:
ocr_df

,index,xml_index,text_top,text_left,text_width,text_height,text,font_size,font_family,font_color,attrib,children,avg_line_height,level_0,word_coords
0,0,0,139,447,9,20,Cont.P.(MD)No.123 of 2020,18,BAAAAA+Thorndale,#000000,HEADER,None,20,NaN,"{'Cont.P.(MD)No.123': {'left': 6, 'conf': 66, ..."
0,NaN,1,181,170,561,21,Cont.P.(MD)No.123 of 2020,18,CAAAAA+BitstreamVeraSerif,#000000,NaN,None,21,NaN,"{'Cont.P.(MD)No.123': {'left': 6, 'conf': 66, ..."
1,NaN,2,223,357,187,21,in,18,CAAAAA+BitstreamVeraSerif,#000000,NaN,None,21,NaN,"{'in': {'left': 4, 'conf': 96, 'top': 8, 'widt..."
2,NaN,3,276,411,80,21,"Crl.0.P.(MD)No.10790 of 2019,",18,CAAAAA+BitstreamVeraSerif,#000000,NaN,None,21,NaN,"{'Crl.0.P.(MD)No.10790': {'left': 6, 'conf': 7..."
2,4,4,318,153,596,21,"Crl.0.P.(MD)No.10790 of 2019,",18,DAAAAA+BitstreamVeraSerif,#000000,NaN,None,21,NaN,"{'Crl.0.P.(MD)No.10790': {'left': 6, 'conf': 7..."
0,NaN,6,361,325,252,21,Cont.P.(MD)No.123 of 2020,18,CAAAAA+BitstreamVeraSerif,#000000,NaN,None,21,NaN,"{'Cont.P.(MD)No.123': {'left': 6, 'conf': 66, ..."
1,NaN,7,403,440,23,21,in,18,CAAAAA+BitstreamVeraSerif,#000000,NaN,None,21,NaN,"{'in': {'left': 4, 'conf': 96, 'top': 8, 'widt..."
2,NaN,8,445,311,279,21,"Crl.0.P.(MD)No.10790 of 2019,",18,CAAAAA+BitstreamVeraSerif,#000000,NaN,None,21,NaN,"{'Crl.0.P.(MD)No.10790': {'left': 6, 'conf': 7..."
3,NaN,9,519,139,83,21,RJayaraj,18,CAAAAA+BitstreamVeraSerif,#000000,NaN,None,21,NaN,"{'RJayaraj': {'left': 6, 'conf': 72, 'top': 8,..."
4,NaN,10,519,618,113,21,http://www. judis.nic.in,18,CAAAAA+BitstreamVeraSerif,#000000,NaN,None,21,NaN,"{'http://www.': {'left': 5, 'conf': 0, 'top': ..."


In [18]:
#draw_box(img_filepath, page_width, page_height,p_df, color='red')

# Draw bounding box for json response 

In [24]:
from src.services import main

In [10]:
pdf_path = input_dir+"/"+filename
save_path  = save_dir+"/"+str(filename.split('.pdf')[0])
if not os.path.exists(save_path):
    os.system('mkdir -p {0}'.format(save_path))
data = main.DocumentStructure(100,filename, None,'eng')

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [112]:
response = data['result']

In [128]:
def draw_bbox_image(draw,page_data):
    
    if page_data['images']:
        try:
            for image_block in page_data['images']:
                    top     = image_block["text_top"];         left   = image_block["text_left"];  
                    bottom  = top+image_block["text_height"];  right  = left+image_block["text_width"]
                    draw.rectangle(((left, top), (right,bottom)), outline='green')
            return draw
        except:
            pass

In [133]:
def draw_bbox_table(draw,page_data):
    
    if page_data['tables']:
        try:
            for table_block in page_data['tables']:
                    top     = table_block["text_top"];         left   = table_block["text_left"];  
                    bottom  = top+table_block["text_height"];  right  = left+table_block["text_width"]
                    draw.rectangle(((left, top), (right,bottom)), outline='blue')
            return draw
        except:
            pass

In [134]:
def draw_bbox_text(draw,page_data):
    
    if page_data['text_blocks']:
        try:
            for text_block in page_data['text_blocks']:
                top     = text_block["text_top"];         left   = text_block["text_left"];  
                bottom  = top+text_block["text_height"];  right  = left+text_block["text_width"]
                draw.rectangle(((left, top), (right,bottom)), outline='red')
            return draw
        except:
            pass

In [135]:
def draw_bbox(data,image_files,save_path):
    for page_no in range(len(data)):
        image_path = sorted(image_files)[page_no]
        page_data = data[page_no]
        page_width = page_data['page_width']; page_height = page_data['page_height']
        image  = Image.open(image_path)
        image  = image.resize((page_width, page_height))
        draw   = ImageDraw.Draw(image)
        draw  = drwa_bbox_text(draw,page_data)
        draw  = drwa_bbox_image(draw,page_data)
        draw  = drwa_bbox_table(draw,page_data)
        save_filepath = os.path.join(save_path,image_path.split('images/')[1])
        image.save(save_filepath)
            

In [136]:
draw_bbox(response,pdf_image_paths,save_path)